In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from transitions import Machine #libarry used for the state machine
import serial
import serial.tools.list_ports
import speech_recognition as sr
import time
#my custom modules imports
import settingsHD as glb #this import all the global variables
import hanoi_img_procHD as vidpro
import matrix_proc as mtrx
#import statemachine as FSMrobot
import voicerecog as vr
import drawing_frames as draw


#-------global variables---

#some global variables
sort_disc_dictionary = {'sort0' : None, 'sort1': None, 'sort2': None}


#variables used for drawing

#states images with theoric matrix
mtx_state_dict = {} #dictionary used to store the images
img_quant = 8  #number of picture states


for i in range (0, (img_quant)):

    name_file = 'DATA/states_img/state' + str(i) + '.jpg'
    name_var = str('state'+str(i))
    mtx_state_dict[name_var] = cv2.imread(name_file)


In [2]:
#serial port initalization with the Arduino UNO
serial_inst = serial.Serial()
serial_inst.baudrate = 115200
port_var = f'COM3' # the number 3 its the Arduino
serial_inst.port = port_var 
serial_inst.open()


In [3]:
# this state machine corresponds to all the behaviour of the Hanoi Droid robot, focused on solve the tower of hanoi game.

#from transitions import Machine

class HanoiDroidFSM(object):

    # Define some states. Most of the time, narcoleptic superheroes are just like
    # everyone else. Except for...
    states = ['zero', 'face_track', 'voice_cmd', 'hanoi_game', 'hg_step1', 'hg_step2', 'hg_step3', 'hg_step4', 'hg_step5', 'hg_step6', 'hg_step7', 'game_solved', 'prompt_error'  ]
              
    # A more compact version of the quickstart transitions
    transitions = [{'trigger': 'voice', 'source': 'zero', 'dest':  'voice_cmd', 'before': 'recognize_command'},
                   {'trigger': 'start_facetrack', 'source': 'voice_cmd', 'dest':  'face_track'},
                   {'trigger': 'timer_facetrack', 'source': 'face_track', 'dest':  'zero'},
                   {'trigger': 'non_command', 'source': 'voice_cmd', 'dest':  'zero'},
                   #{'trigger': 'custom_disc', 'source': 'voice_cmd', 'dest':  'custom_disc'}, future deployment
                   #{'trigger': 'get_mtx', 'source': 'custom_disc', 'dest':  'organize_mov'}, future deployment
                   {'trigger': 'ack', 'source': 'organize_mov', 'dest':  'comp_matrix'},
                   {'trigger': 'eq_mtx', 'source': 'comp_matrix', 'dest':  'game_solved'},
                   {'trigger': 'dif_mtx', 'source': 'comp_matrix', 'dest':  'prompt_error'},
                   {'trigger': 'solve_hanoi', 'source': 'voice_cmd', 'dest':  'hanoi_game'},
                   {'trigger': 'hg_start', 'source': 'hanoi_game', 'dest':  'hg_step1', 'before': 'serial_cmd_step1'},
                   {'trigger': 'ack', 'source': 'hg_step1', 'dest':  'hg_step2', 'before': 'serial_cmd_step2'},
                   {'trigger': 'ack', 'source': 'hg_step2', 'dest':  'hg_step3', 'before': 'serial_cmd_step3'},
                   {'trigger': 'ack', 'source': 'hg_step3', 'dest':  'hg_step4', 'before': 'serial_cmd_step4'},
                   {'trigger': 'ack', 'source': 'hg_step4', 'dest':  'hg_step5', 'before': 'serial_cmd_step5'},
                   {'trigger': 'ack', 'source': 'hg_step5', 'dest':  'hg_step6', 'before': 'serial_cmd_step6'},
                   {'trigger': 'ack', 'source': 'hg_step6', 'dest':  'hg_step7', 'before': 'serial_cmd_step7'},
                   {'trigger': 'dif_mtx', 'source': 'hanoi_game', 'dest':  'prompt_error'},
                   {'trigger': 'dif_mtx', 'source': 'hg_step1', 'dest':  'prompt_error', 'before': 'serial_cmd_rest'},
                   {'trigger': 'dif_mtx', 'source': 'hg_step2', 'dest':  'prompt_error', 'before': 'serial_cmd_rest'},
                   {'trigger': 'dif_mtx', 'source': 'hg_step3', 'dest':  'prompt_error', 'before': 'serial_cmd_rest'},
                   {'trigger': 'dif_mtx', 'source': 'hg_step4', 'dest':  'prompt_error', 'before': 'serial_cmd_rest'},
                   {'trigger': 'dif_mtx', 'source': 'hg_step5', 'dest':  'prompt_error', 'before': 'serial_cmd_rest'},
                   {'trigger': 'dif_mtx', 'source': 'hg_step6', 'dest':  'prompt_error', 'before': 'serial_cmd_rest'},
                   {'trigger': 'dif_mtx', 'source': 'hg_step7', 'dest':  'prompt_error', 'before': 'serial_cmd_rest'},
                   {'trigger': 'ack', 'source': 'hg_step7', 'dest':  'game_solved', 'before': 'serial_cmd_rest'},
                   {'trigger': 'timer_show', 'source': 'game_solved', 'dest':  'zero'},
                   {'trigger': 'timer_show', 'source': 'prompt_error', 'dest':  'zero'}]
    
    
    
    def __init__(self, name):

        self.name = name
        #self.kittens_rescued = 0  # What have we accomplished today?
        self.command = 'none'
       
        # Initialize the state machine
        self.machine = Machine(model=self, states=HanoiDroidFSM.states,
                               transitions=HanoiDroidFSM.transitions, initial='zero')
        
            
    
    def recognize_command(self):
        print("I am in Voice state")
        
    def serial_cmd_step1(self):
        command = 'a'
        serial_inst.write(command.encode('utf-8'))
        time.sleep(27)
       
    def serial_cmd_step2(self):
        command = 'b'
        serial_inst.write(command.encode('utf-8'))
        time.sleep(25)
        
    def serial_cmd_step3(self):
        command = 'c'
        serial_inst.write(command.encode('utf-8'))
        time.sleep(25)

    def serial_cmd_step4(self):
        command = 'd'
        serial_inst.write(command.encode('utf-8'))
        time.sleep(25)
        
    def serial_cmd_step5(self):
        command = 'e'
        serial_inst.write(command.encode('utf-8'))
        time.sleep(25)

    def serial_cmd_step6(self):
        command = 'f'
        serial_inst.write(command.encode('utf-8'))
        time.sleep(25)
         
    def serial_cmd_step7(self):
        command = 'g'
        serial_inst.write(command.encode('utf-8'))
        time.sleep(27)
        
    def serial_cmd_rest(self):
        command = 'R'
        serial_inst.write(command.encode('utf-8'))
        time.sleep(10)
        
        

In [4]:
#create the state machine object

#droid = FSMrobot.HanoiDroidFSM("Droid") remove comment for final
droid = HanoiDroidFSM("Droid")



In [5]:
while True:

    
#-------------STATE-------------    
    while droid.state == 'zero':
    
        window = draw.draw_intro()
        # Display the frame with the four images
        cv2.imshow('MainWindow', window)
        cv2.waitKey(1)

        print ("inside zero state")
        have_a_command, command = vr.start_robot_voice_command()
        if have_a_command:
            droid.voice()
            break

#-------------STATE------------- 
    if droid.state == 'voice_cmd':
        if "start" in command:
            print("Starting the hanoi solving...")
            droid.solve_hanoi()
    
        elif "facetrack" in command:
            print("Starting the hanoi solving...")
            droid.start_facetrack()
    
        else:
            print("command not in my instructions")
            droid.non_command()     
        
#-------------STATE------------- 

    elif droid.state == 'hanoi_game':
        print ("inside hanoi_game state")
    

        frame = vidpro.take_one_frame() #this is to take the picture of the outside. I should compare to the inital state
        small_image_hsv = vidpro.convert_and_crop(frame)
    
    
        #pilas: cuando se trabaje con el definitivo, tengo que cambiar el nombre de imagecopy por el de frame
        for i in range (0, 3):
            image2 = cv2.rectangle(frame, (glb.mask_x_position_dict['x'+str(i)], glb.mask_y_position_dict['y'+str(i)]+130), ((glb.mask_x_position_dict['x'+str(i)] + glb.mask_width), (glb.mask_y_position_dict['y'+str(i)]+glb.mask_height+130)), (0,0,255), thickness = 2)
  
        frame3 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    
        #display(image2)
    
        for i in range(0, 3):
            imagehsv_roi = small_image_hsv[glb.mask_y_position_dict['y'+str(i)]:(glb.mask_y_position_dict['y'+str(i)] + glb.mask_height), int(glb.mask_x_position_dict['x'+str(i)]):int(glb.mask_x_position_dict['x'+str(i)]+glb.mask_width)]
            #display(imagehsv_roi)
            sort_disc_dictionary['sort' + str(i)] = vidpro.detect_color(imagehsv_roi) 
    
        state = 0
        frame2 = mtx_state_dict['state' + str(state)]
        #hanoi_mat= glb.hanoi_matrix_char[(3*state):(3*(state+1)), :] #this is the theorical correct matrix
        detected_matrix_char = mtrx.get_detected_matrix_char(sort_disc_dictionary) #this is the matrix detected by the camera
        frame1 = draw.write_in_image(image2,detected_matrix_char) #write the letters of the detected matrix
        #flip_detected_matrix_char = np.fliplr(detected_matrix_char) #need to flip to adjust user/robot perspective
        matrices_are_equal = mtrx.compare_matrix (detected_matrix_char, state)
    
        #display(image_with_letters)
    
        if matrices_are_equal: #if the matrices are equal, continues next state, if not, go to prompt error
            frame4 = cv2.imread('DATA/states_img/match.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(6) #time to show and explain
            command = 'S'
            serial_inst.write(command.encode('utf-8'))
            time.sleep(3) #give time to raise the hand
            droid.hg_start()
    
        else:
            frame4 = cv2.imread('DATA/states_img/nomatch.png')    
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)        
            time.sleep(6)
            droid.dif_mtx()
          

        
    #-------------STATE-------------         

        
    elif droid.state == 'hg_step1':
        print ("inside step1 state")
    

        frame = vidpro.take_one_frame() #this is to take the picture of the outside. I should compare to the inital state
        small_image_hsv = vidpro.convert_and_crop(frame)
    
    
        #pilas: cuando se trabaje con el definitivo, tengo que cambiar el nombre de imagecopy por el de frame
        for i in range (0, 3):
            image2 = cv2.rectangle(frame, (glb.mask_x_position_dict['x'+str(i)], glb.mask_y_position_dict['y'+str(i)]+130), ((glb.mask_x_position_dict['x'+str(i)] + glb.mask_width), (glb.mask_y_position_dict['y'+str(i)]+glb.mask_height+130)), (0,0,255), thickness = 2)
  
        frame3 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    
        #display(image2)
    
        for i in range(0, 3):
            imagehsv_roi = small_image_hsv[glb.mask_y_position_dict['y'+str(i)]:(glb.mask_y_position_dict['y'+str(i)] + glb.mask_height), int(glb.mask_x_position_dict['x'+str(i)]):int(glb.mask_x_position_dict['x'+str(i)]+glb.mask_width)]
            #display(imagehsv_roi)
            sort_disc_dictionary['sort' + str(i)] = vidpro.detect_color(imagehsv_roi) 
    
        state = 1  #this variable change every time based on the state that is inside  
        frame2 = mtx_state_dict['state' + str(state)]
        detected_matrix_char = mtrx.get_detected_matrix_char(sort_disc_dictionary) #this is the matrix detected by the camera
        frame1 = draw.write_in_image(image2,detected_matrix_char) #write the letters of the detected matrix
        matrices_are_equal = mtrx.compare_matrix (detected_matrix_char, state)
    
        #display(image_with_letters)
    
        if matrices_are_equal: #if the matrices are equal, continues next state, if not, go to prompt error
            frame4 = cv2.imread('DATA/states_img/match.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(10) #time to show and explain
            droid.ack()
        else:
        
            frame4 = cv2.imread('DATA/states_img/nomatch.png')    
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(4) #time to show and explain
            droid.dif_mtx()   
       
    
        #-------------STATE------------- 

    elif droid.state == 'hg_step2':
        print ("inside step2 state")
    
        #--------- UNCOMMENT FOR FINAL APPLICATION
        frame = vidpro.take_one_frame() #this is to take the picture of the outside. I should compare to the inital state
        small_image_hsv = vidpro.convert_and_crop(frame)
     

        #pilas: cuando se trabaje con el definitivo, tengo que cambiar el nombre de imagecopy por el de frame
        for i in range (0, 3):
            image2 = cv2.rectangle(frame, (glb.mask_x_position_dict['x'+str(i)], glb.mask_y_position_dict['y'+str(i)]+130), ((glb.mask_x_position_dict['x'+str(i)] + glb.mask_width), (glb.mask_y_position_dict['y'+str(i)]+glb.mask_height+130)), (0,0,255), thickness = 2)
  
        frame3 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    
        #display(image2)
    
        for i in range(0, 3):
            imagehsv_roi = small_image_hsv[glb.mask_y_position_dict['y'+str(i)]:(glb.mask_y_position_dict['y'+str(i)] + glb.mask_height), int(glb.mask_x_position_dict['x'+str(i)]):int(glb.mask_x_position_dict['x'+str(i)]+glb.mask_width)]
            #display(imagehsv_roi)
            sort_disc_dictionary['sort' + str(i)] = vidpro.detect_color(imagehsv_roi) 
    
        state = 2  #this variable change every time based on the state that is inside  
        frame2 = mtx_state_dict['state' + str(state)]
        detected_matrix_char = mtrx.get_detected_matrix_char(sort_disc_dictionary) #this is the matrix detected by the camera
        frame1 = draw.write_in_image(image2,detected_matrix_char) #write the letters of the detected matrix
        matrices_are_equal = mtrx.compare_matrix (detected_matrix_char, state)
    
        #display(image_with_letters)
    
        if matrices_are_equal: #if the matrices are equal, continues next state, if not, go to prompt error
            frame4 = cv2.imread('DATA/states_img/match.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(5) #time to show and explain
            droid.ack()
        else:
        
            frame4 = cv2.imread('DATA/states_img/nomatch.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(4) #time to show and explain
            droid.dif_mtx()
        
        
        
        #-------------STATE------------- 
        
    elif droid.state == 'hg_step3':
        print ("inside step3 state")
    
        #--------- UNCOMMENT FOR FINAL APPLICATION
        frame = vidpro.take_one_frame() #this is to take the picture of the outside. I should compare to the inital state
        small_image_hsv = vidpro.convert_and_crop(frame)
    
    
        #pilas: cuando se trabaje con el definitivo, tengo que cambiar el nombre de imagecopy por el de frame
        for i in range (0, 3):
            image2 = cv2.rectangle(frame, (glb.mask_x_position_dict['x'+str(i)], glb.mask_y_position_dict['y'+str(i)]+130), ((glb.mask_x_position_dict['x'+str(i)] + glb.mask_width), (glb.mask_y_position_dict['y'+str(i)]+glb.mask_height+130)), (0,0,255), thickness = 2)
  
        frame3 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    
        #display(image2)
    
        for i in range(0, 3):
            imagehsv_roi = small_image_hsv[glb.mask_y_position_dict['y'+str(i)]:(glb.mask_y_position_dict['y'+str(i)] + glb.mask_height), int(glb.mask_x_position_dict['x'+str(i)]):int(glb.mask_x_position_dict['x'+str(i)]+glb.mask_width)]
            #display(imagehsv_roi)
            sort_disc_dictionary['sort' + str(i)] = vidpro.detect_color(imagehsv_roi) 
    
        state = 3  #this variable change every time based on the state that is inside  
        frame2 = mtx_state_dict['state' + str(state)]
        detected_matrix_char = mtrx.get_detected_matrix_char(sort_disc_dictionary) #this is the matrix detected by the camera
        frame1 = draw.write_in_image(image2,detected_matrix_char) #write the letters of the detected matrix
        matrices_are_equal = mtrx.compare_matrix (detected_matrix_char, state)
    
        #display(image_with_letters)
    
        if matrices_are_equal: #if the matrices are equal, continues next state, if not, go to prompt error
            frame4 = cv2.imread('DATA/states_img/match.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(2) #time to show and explain
            droid.ack()
        else:
        
            frame4 = cv2.imread('DATA/states_img/nomatch.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(4) #time to show and explain
            droid.dif_mtx()
        
        
        
        #-------------STATE------------- 
        
    elif droid.state == 'hg_step4':
        print ("inside step4 state")
    
        #--------- UNCOMMENT FOR FINAL APPLICATION
        frame = vidpro.take_one_frame() #this is to take the picture of the outside. I should compare to the inital state
        small_image_hsv = vidpro.convert_and_crop(frame)
    
        #pilas: cuando se trabaje con el definitivo, tengo que cambiar el nombre de imagecopy por el de frame
        for i in range (0, 3):
            image2 = cv2.rectangle(frame, (glb.mask_x_position_dict['x'+str(i)], glb.mask_y_position_dict['y'+str(i)]+130), ((glb.mask_x_position_dict['x'+str(i)] + glb.mask_width), (glb.mask_y_position_dict['y'+str(i)]+glb.mask_height+130)), (0,0,255), thickness = 2)
  
        frame3 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    
        #display(image2)
    
        for i in range(0, 3):
            imagehsv_roi = small_image_hsv[glb.mask_y_position_dict['y'+str(i)]:(glb.mask_y_position_dict['y'+str(i)] + glb.mask_height), int(glb.mask_x_position_dict['x'+str(i)]):int(glb.mask_x_position_dict['x'+str(i)]+glb.mask_width)]
            #display(imagehsv_roi)
            sort_disc_dictionary['sort' + str(i)] = vidpro.detect_color(imagehsv_roi) 
    
        state = 4  #this variable change every time based on the state that is inside  
        frame2 = mtx_state_dict['state' + str(state)]
        detected_matrix_char = mtrx.get_detected_matrix_char(sort_disc_dictionary) #this is the matrix detected by the camera
        frame1 = draw.write_in_image(image2,detected_matrix_char) #write the letters of the detected matrix
        matrices_are_equal = mtrx.compare_matrix (detected_matrix_char, state)
    
        #display(image_with_letters)
    
        if matrices_are_equal: #if the matrices are equal, continues next state, if not, go to prompt error
            frame4 = cv2.imread('DATA/states_img/match.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(2) #time to show and explain
            droid.ack()
        else:
        
            frame4 = cv2.imread('DATA/states_img/nomatch.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(4) #time to show and explain
            droid.dif_mtx()
    
        #-------------STATE-------------         
        
    elif droid.state == 'hg_step5':
        print ("inside step5 state")
    
        #--------- UNCOMMENT FOR FINAL APPLICATION
        frame = vidpro.take_one_frame() #this is to take the picture of the outside. I should compare to the inital state
        small_image_hsv = vidpro.convert_and_crop(frame)
    
  
        #pilas: cuando se trabaje con el definitivo, tengo que cambiar el nombre de imagecopy por el de frame
        for i in range (0, 3):
            image2 = cv2.rectangle(frame, (glb.mask_x_position_dict['x'+str(i)], glb.mask_y_position_dict['y'+str(i)]+130), ((glb.mask_x_position_dict['x'+str(i)] + glb.mask_width), (glb.mask_y_position_dict['y'+str(i)]+glb.mask_height+130)), (0,0,255), thickness = 2)
  
        frame3 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    
        #display(image2)
    
        for i in range(0, 3):
            imagehsv_roi = small_image_hsv[glb.mask_y_position_dict['y'+str(i)]:(glb.mask_y_position_dict['y'+str(i)] + glb.mask_height), int(glb.mask_x_position_dict['x'+str(i)]):int(glb.mask_x_position_dict['x'+str(i)]+glb.mask_width)]
            #display(imagehsv_roi)
            sort_disc_dictionary['sort' + str(i)] = vidpro.detect_color(imagehsv_roi) 
    
        state = 5  #this variable change every time based on the state that is inside  
        frame2 = mtx_state_dict['state' + str(state)]
        detected_matrix_char = mtrx.get_detected_matrix_char(sort_disc_dictionary) #this is the matrix detected by the camera
        frame1 = draw.write_in_image(image2,detected_matrix_char) #write the letters of the detected matrix
        matrices_are_equal = mtrx.compare_matrix (detected_matrix_char, state)
        #display(image_with_letters)
    
        if matrices_are_equal: #if the matrices are equal, continues next state, if not, go to prompt error
            frame4 = cv2.imread('DATA/states_img/match.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(2) #time to show and explain
            droid.ack()
        else:
        
            frame4 = cv2.imread('DATA/states_img/nomatch.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(4) #time to show and explain
            droid.dif_mtx()      
        
        
        
        #-------------STATE-------------   
        
    elif droid.state == 'hg_step6':
        print ("inside step6 state")
    
        #--------- UNCOMMENT FOR FINAL APPLICATION
        frame = vidpro.take_one_frame() #this is to take the picture of the outside. I should compare to the inital state
        small_image_hsv = vidpro.convert_and_crop(frame)
    
        #pilas: cuando se trabaje con el definitivo, tengo que cambiar el nombre de imagecopy por el de frame
        for i in range (0, 3):
            image2 = cv2.rectangle(frame, (glb.mask_x_position_dict['x'+str(i)], glb.mask_y_position_dict['y'+str(i)]+130), ((glb.mask_x_position_dict['x'+str(i)] + glb.mask_width), (glb.mask_y_position_dict['y'+str(i)]+glb.mask_height+130)), (0,0,255), thickness = 2)
  
        frame3 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    
        #display(image2)
    
        for i in range(0, 3):
            imagehsv_roi = small_image_hsv[glb.mask_y_position_dict['y'+str(i)]:(glb.mask_y_position_dict['y'+str(i)] + glb.mask_height), int(glb.mask_x_position_dict['x'+str(i)]):int(glb.mask_x_position_dict['x'+str(i)]+glb.mask_width)]
            #display(imagehsv_roi)
            sort_disc_dictionary['sort' + str(i)] = vidpro.detect_color(imagehsv_roi) 
    
        state = 6  #this variable change every time based on the state that is inside  
        frame2 = mtx_state_dict['state' + str(state)]
        detected_matrix_char = mtrx.get_detected_matrix_char(sort_disc_dictionary) #this is the matrix detected by the camera
        frame1 = draw.write_in_image(image2,detected_matrix_char) #write the letters of the detected matrix
        matrices_are_equal = mtrx.compare_matrix (detected_matrix_char, state)
    
        #display(image_with_letters)
    
        if matrices_are_equal: #if the matrices are equal, continues next state, if not, go to prompt error
            frame4 = cv2.imread('DATA/states_img/match.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(2) #time to show and explain
            droid.ack()
        else:
        
            frame4 = cv2.imread('DATA/states_img/nomatch.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(4) #time to show and explain
            droid.dif_mtx()
  
      
            #-------------STATE------------- 
    
    elif droid.state == 'hg_step7':
        print ("inside step7 state")
    
        #--------- UNCOMMENT FOR FINAL APPLICATION
        frame = vidpro.take_one_frame() #this is to take the picture of the outside. I should compare to the inital state
        small_image_hsv = vidpro.convert_and_crop(frame)
   

        #pilas: cuando se trabaje con el definitivo, tengo que cambiar el nombre de imagecopy por el de frame
        for i in range (0, 3):
            image2 = cv2.rectangle(frame, (glb.mask_x_position_dict['x'+str(i)], glb.mask_y_position_dict['y'+str(i)]+130), ((glb.mask_x_position_dict['x'+str(i)] + glb.mask_width), (glb.mask_y_position_dict['y'+str(i)]+glb.mask_height+130)), (0,0,255), thickness = 2)

        frame3 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)

        #display(image2)

        for i in range(0, 3):
            imagehsv_roi = small_image_hsv[glb.mask_y_position_dict['y'+str(i)]:(glb.mask_y_position_dict['y'+str(i)] + glb.mask_height), int(glb.mask_x_position_dict['x'+str(i)]):int(glb.mask_x_position_dict['x'+str(i)]+glb.mask_width)]
            #display(imagehsv_roi)
            sort_disc_dictionary['sort' + str(i)] = vidpro.detect_color(imagehsv_roi) 

        state = 7  #this variable change every time based on the state that is inside  
        frame2 = mtx_state_dict['state' + str(state)]
        detected_matrix_char = mtrx.get_detected_matrix_char(sort_disc_dictionary) #this is the matrix detected by the camera
        frame1 = draw.write_in_image(image2,detected_matrix_char) #write the letters of the detected matrix
        matrices_are_equal = mtrx.compare_matrix (detected_matrix_char, state)

        #display(image_with_letters)

        if matrices_are_equal: #if the matrices are equal, continues next state, if not, go to prompt error
            frame4 = cv2.imread('DATA/states_img/match.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(2) #time to show and explain
            droid.ack()
        else:

            frame4 = cv2.imread('DATA/states_img/nomatch.png')
            window = draw.draw_matrix(frame1,frame2,frame3,frame4)
            cv2.imshow('MainWindow', window)
            cv2.waitKey(1)
            time.sleep(4) #time to show and explain
            droid.dif_mtx()

            #-------------STATE-------------  
            
            
    elif droid.state == 'prompt_error':
        print ("inside the error prompt state")
        window = draw.draw_error()
        cv2.imshow('MainWindow', window)
        cv2.waitKey(1)
        time.sleep(5)
        droid.timer_show()

    
        #-------------STATE-------------  
    
    elif droid.state == 'game_solved':
        print ("inside the error prompt state")
        window =  draw.draw_solved()
        cv2.imshow('MainWindow', window)
        cv2.waitKey(1)
        time.sleep(5)
        droid.timer_show()

    

inside zero state
Listening for 'robot start' command...
result2:
{   'alternative': [   {'confidence': 0.85393143, 'transcript': 'start'},
                       {'confidence': 0.85393143, 'transcript': 'starte'},
                       {'confidence': 0.85393143, 'transcript': 'startt'},
                       {'confidence': 0.85393143, 'transcript': 'staart'},
                       {'confidence': 0.43744484, 'transcript': 'start it'}],
    'final': True}
I am in Voice state
Starting the hanoi solving...
inside hanoi_game state
Frame taken
inside step1 state
Frame taken
inside step2 state
Frame taken
inside step3 state
Frame taken
inside the error prompt state
inside zero state
Listening for 'robot start' command...
result2:
{   'alternative': [   {'confidence': 0.90595084, 'transcript': 'start'},
                       {'confidence': 0.90595084, 'transcript': 'starte'},
                       {'confidence': 0.90595084, 'transcript': 'startt'},
                       {'confidence': 0

UnboundLocalError: local variable 'frame' referenced before assignment